Follow the steps below to implement DEA primary CCR input-oriented model.

The analysed problem is the following: we have set of 4 car garages with 2 inputs and 2 outputs. We need to identify which ones of them are efficient.

1) Load necessry package

In [1]:
library(lpSolveAPI)

2) Load data about input and output values from files inputs-ex2.csv and outputs-ex2.csv

Use read.csv function as in Execrise 1.

In [2]:
inputs <- read.csv("inputs-ex2.csv", header = TRUE, sep=";")
outputs <- read.csv("outputs-ex2.csv", header = TRUE, sep=";")
inputs
outputs

X,Staff,Showroom.space
<fct>,<int>,<int>
Winchester,7,8
Andover,6,6
Basingstoke,2,3
Poole,14,9


X,Sales,Profit
<fct>,<dbl>,<dbl>
Winchester,2.0,1.5
Andover,2.3,1.6
Basingstoke,0.8,0.5
Poole,2.6,1.9


3) Before implementing the model, we remind the primary CCR model for calculating efficiency score for a selected car garage.

Objective function:

$$max \space μ_1y_{10} + μ_2y_{20}$$

where $y_{10}$ and $y_{20}$ are, respectively, values of outputs for garage for which efficiency score we want to calculate.

Constraints:
* weighted sum of inputs for a subject unit equal to 1:
$$ν_1x_{10} + ν_2x_{20} = 1$$
* efficiencies for all units are <= 1:
$$ μ_1y_{1i} + μ_2y_{2i} <= ν_1x_{1i} + ν_2x_{2i}, \space i = 1, 2, ..., K$$ (K=4 in our case)

4) Implement body of createModel function. 

Return the model using make.lp function (as in exercise 1).

How many decision variables do you need in the problem?

In [3]:
createModel = function()
{
    param = ncol(inputs) + ncol(outputs) - 2
    return(make.lp(ncol = param)) #variables 1-2 input weights, 3-4 output weights
}
model <- createModel()

5) Set model's optimization direction (as in the exercise before). 

You don't need to change variable types. They are of type "real" by default.

In [4]:
setDirection = function(model) 
{
    lp.control(model, sense = "max")
}
setDirection(model)

$anti.degen
[1] "fixedvars" "stalling" 

$basis.crash
[1] "none"

$bb.depthlimit
[1] -50

$bb.floorfirst
[1] "automatic"

$bb.rule
[1] "pseudononint" "greedy"       "dynamic"      "rcostfixing" 

$break.at.first
[1] FALSE

$break.at.value
[1] 1e+30

$epsilon
      epsb       epsd      epsel     epsint epsperturb   epspivot 
     1e-10      1e-09      1e-12      1e-07      1e-05      2e-07 

$improve
[1] "dualfeas" "thetagap"

$infinite
[1] 1e+30

$maxpivot
[1] 250

$mip.gap
absolute relative 
   1e-11    1e-11 

$negrange
[1] -1e+06

$obj.in.basis
[1] TRUE

$pivoting
[1] "devex"    "adaptive"

$presolve
[1] "none"

$scalelimit
[1] 5

$scaling
[1] "geometric"   "equilibrate" "integers"   

$sense
[1] "maximize"

$simplextype
[1] "dual"   "primal"

$timeout
[1] 0

$verbose
[1] "neutral"

6) Add model objective function.

Implement setObjective function body. The arguments are: created model, loaded files (inputs and outputs) and identifier of garage which efficiency is to be computed.

Set coefficients for the objective function, i.e., output values for the garage under evaluation. Additionally, you need to set which indices (columns -> 3 and 4 as these are outputs) take part in the objective function (indices = c(3, 4)).

In [5]:
setObjective = function(model, inputs, outputs, subjectGarageId)
{
    objective_coeffs = outputs[subjectGarageId, 2:3]
    set.objfn(model, objective_coeffs, indices = c(3,4)) 
}
setObjective(model, inputs, outputs, 1)

7) Add model constraint which sets the weighted sum of inputs of the considered garage to 1.

In [6]:
addUnityConstraint = function(model, inputs, outputs, subjectGarageId)
{
    constraint_coeffs = array(0, dim=4)
    constraint_coeffs[1:2] = unlist(inputs[subjectGarageId, 2:3])
    add.constraint(model, constraint_coeffs, "=", 1) 
}
addUnityConstraint(model, inputs, outputs, 1)

8) Add inequality constraints (limiting the efficiency of each garage to 1).

In [7]:
addOtherConstraints = function(model, inputs, outputs)
{
    for(i in 1:nrow(inputs)) # one constraint for each DMU
    {
        constraint_coeffs = array(0, dim = 4)
        constraint_coeffs[1:2] = unlist(-1 * inputs[i, 2:3])
        constraint_coeffs[3:4] = unlist(outputs[i, 2:3])
        add.constraint(model, constraint_coeffs, "<=", 0)
    }
}
addOtherConstraints(model, inputs, outputs)

9) Run the solver and print the efficiency score 

In [8]:
solve(model)
get.objective(model)

[1] 0

[1] 0.8035714

In [9]:
inputs <- read.csv("inputs-ex2.csv", sep=";")
outputs <- read.csv("outputs-ex2.csv", sep=";")
for(garage in 1:4)
{
    model = createModel()
    setDirection(model)
    setObjective(model, inputs, outputs, garage)
    addUnityConstraint(model, inputs, outputs, garage)
    addOtherConstraints(model, inputs, outputs)
    solve(model)
    efficiency = get.objective(model)
    print(paste(inputs[garage, 1], ": ", efficiency, sep=""))
}

[1] "Winchester: 0.803571428571429"
[1] "Andover: 1"
[1] "Basingstoke: 1"
[1] "Poole: 0.791666666666667"
